In [1]:
!pip install PyPDF2 pytesseract pdf2image pillow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 7.7 MB/s eta 0:00:00


In [2]:
!sudo apt-get install tesseract-ocr tesseract-ocr-ara

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
tesseract-ocr is already the newest version (4.1.1-2.1build1).
The following NEW packages will be installed:
  tesseract-ocr-ara
0 upgraded, 1 newly installed, 0 to remove and 34 not upgraded.
Need to get 645 kB of archives.
After this operation, 1,447 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-ara all 1:4.00~git30-7274cfa-1.1 [645 kB]
Fetched 645 kB in 1s (537 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 78, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 
Selecting p

In [3]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.4/491.4 kB 15.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 17.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 18.5 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system 

In [18]:
from datasets import load_dataset

# Load the full corpus
dataset = load_dataset("mohres/The_Arabic_E-Book_Corpus", split="train")

# Print metadata for the first few books to choose one
for i in range(100):
    print(dataset[i]['title'], dataset[i]['author'], dataset[i]["booknr"])

لؤلؤة الحب هربرت جورج ويلز 13028368
مغامرة العميل المرموق آرثر كونان دويل 13035305
ثلاثة دروس في ديكارت ألكسندر كواريه 13050381
كشف المُخبَّا عن فنون أوربا أحمد فارس الشدياق 13068491
حقائق الإسلام وأباطيل خصومه عباس محمود العقاد 13082050
مدينة زحلة عيسى إسكندر المعلوف 13149739
لاسلكيًّا روديارد كبلينج 13153615
مغامرة راكبة الدراجة في الطريق المهجور آرثر كونان دويل 13181417
الفاروق عمر محمد حسين هيكل 13538483
إبراهيم الثاني إبراهيم عبد القادر المازني 13572470
الطبيعة وما بعد الطبيعة: المادة . الحياة . الله يوسف كرم 13572791
الحجاج بن يوسف جُرجي زيدان 13595164
المسألة السودانية عمر طوسون 13603829
غَادَةُ كرْبَلاء جُرجي زيدان 13604281
أبيض الناب جاك لندن 13639135
أنت من فعلها! إدجار آلان بو 13649293
البدوية إبراهيم رمزي 13680727
قصة الأيام القادمة هربرت جورج ويلز 13736816
ألف ليلة وليلة ألف ليلة وليلة 13815802
نماذج بشرية أحمد رضا حوحو 13919315
تذكار الصبا: ذكرى ١٩ مارس محمد لطفي جمعة 13919490
الإسلام والحضارة الإنسانية عباس محمود العقاد 13920718
رواية الشقيقتين هنري لامنس 13925094
فَلسَف

In [5]:
x =dataset["booknr"]
i=0
for nr in dataset["booknr"]:
  if nr == 84858130:
    book = dataset[i]
    text = book['text']
    # Save the text to a file
    with open("selected_book.txt", "w", encoding="utf-8") as f:
        f.write(text)
  i +=1

In [6]:
# Install required packages
!pip install PyPDF2 pytesseract pdf2image

# Download and install tesseract + Arabic language pack if running in Colab
import os
if 'google.colab' in str(get_ipython()):
    !apt-get install tesseract-ocr
    !apt-get install tesseract-ocr-ara
    !apt-get install poppler-utils

import re
import json
import PyPDF2
import pytesseract
from pdf2image import convert_from_path
import numpy as np
from PIL import Image
import tempfile
from google.colab import files

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
tesseract-ocr is already the newest version (4.1.1-2.1build1).
0 upgraded, 0 newly installed, 0 to remove and 34 not upgraded.
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
tesseract-ocr-ara is already the newest version (1:4.00~git30-7274cfa-1.1).
0 upgraded, 0 newly installed, 0 to remove and 34 not upgraded.
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  poppler-utils
0 upgraded, 1 newly installed, 0 to remove and 34 not upgraded.
Need to get 186 kB of archives.
After this operation, 697 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 poppler-utils amd64 22.02.0-2ubuntu0.8 [186 kB]
Fetched 186 kB in 1s (221 kB/s)
Selecting previously unselected package poppler-utils.
(Reading database

In [7]:
import os
import re
import json
try:
    from google.colab import files
except ImportError:
    files = None
import PyPDF2
from pdf2image import convert_from_path
import pytesseract

def extract_text_from_pdf(pdf_path, use_ocr=True):
    """
    Extract text from a PDF file.
    If use_ocr is True, uses OCR (recommended for Arabic PDFs).
    Otherwise, tries to extract text directly (might not work well with Arabic).
    """
    if not use_ocr:
        # Try to extract text directly from PDF
        text = ""
        with open(pdf_path, 'rb') as file:
            pdf_reader = PyPDF2.PdfReader(file)
            for page_num in range(len(pdf_reader.pages)):
                page = pdf_reader.pages[page_num]
                text += page.extract_text() + "\n\n"
        return text

    # If OCR is requested or direct extraction didn't yield good results
    print("Using OCR to extract text from PDF...")

    # Convert PDF to images
    print("Converting PDF to images...")
    images = convert_from_path(pdf_path)

    # Extract text from each image using OCR with Arabic language
    full_text = ""
    for i, image in enumerate(images):
        print(f"Processing page {i+1}/{len(images)}...")
        text = pytesseract.image_to_string(image, lang='ara')  # 'ara' for Arabic
        full_text += text + "\n\n"

    return full_text

def extract_text_from_txt(txt_path):
    """Extract text from a .txt file."""
    print(f"Extracting text from {txt_path}...")
    with open(txt_path, 'r', encoding='utf-8') as file:
        text = file.read()
    return text

def split_into_paragraphs(text):
    """Split text into paragraphs based on new lines."""
    paragraphs = re.split(r'\n+', text)
    return [p.strip() for p in paragraphs if p.strip()]

def split_into_sentences(paragraph):
    """Split an Arabic paragraph into sentences."""
    sentences = re.split(r'([.!?؟।\n]+)', paragraph)
    processed_sentences = []
    for i in range(0, len(sentences)-1, 2):
        if i+1 < len(sentences):
            processed_sentences.append(sentences[i] + sentences[i+1])
        else:
            processed_sentences.append(sentences[i])
    if len(sentences) % 2 == 1 and sentences[-1].strip():
        processed_sentences.append(sentences[-1])
    return [s.strip() for s in processed_sentences if s.strip()]

def chunk_paragraphs(paragraphs, min_sentences=2, max_sentences=4):
    """
    Split paragraphs into chunks of 2-4 sentences.
    Returns a list of chunks.
    """
    chunks = []
    for paragraph in paragraphs:
        sentences = split_into_sentences(paragraph)
        if len(sentences) <= min_sentences:
            chunks.append(" ".join(sentences))
            continue
        current_chunk = []
        for sentence in sentences:
            current_chunk.append(sentence)
            if len(current_chunk) >= max_sentences:
                chunks.append(" ".join(current_chunk))
                current_chunk = []
        if current_chunk:
            chunks.append(" ".join(current_chunk))
    return chunks

def process_arabic_book(file_path, output_file='chunked_arabic_book.json', use_ocr=True):
    """Process an Arabic PDF or .txt book and save chunks to a JSON file."""
    # Determine file type and extract text
    file_extension = os.path.splitext(file_path)[1].lower()
    if file_extension == '.pdf':
        text = extract_text_from_pdf(file_path, use_ocr)
    elif file_extension == '.txt':
        text = extract_text_from_txt(file_path)
    else:
        raise ValueError("Unsupported file type. Please provide a .pdf or .txt file.")

    # Save extracted text
    text_file = os.path.splitext(output_file)[0] + "_full_text.txt"
    with open(text_file, 'w', encoding='utf-8') as f:
        f.write(text)
    print(f"Full extracted text saved to {text_file}")

    # Process text into paragraphs and chunks
    paragraphs = split_into_paragraphs(text)
    chunks = chunk_paragraphs(paragraphs)

    # Save output to JSON
    output_data = {
        "source": os.path.basename(file_path),
        "chunks": [{"id": i, "text": chunk} for i, chunk in enumerate(chunks)]
    }
    with open(output_file, 'w', encoding='utf-8') as f:
        json.dump(output_data, f, ensure_ascii=False, indent=2)

    print(f"Processed {len(paragraphs)} paragraphs into {len(chunks)} chunks")
    print(f"Output saved to {output_file}")
    return chunks, text_file, output_file

# Handle input based on environment
if files is not None:
    # Running in Google Colab
    print("Running in Google Colab environment")
    print("Would you like to:")
    print("1. Upload the Arabic book file (PDF or TXT)")
    print("2. Enter the file path to the Arabic book file")
    choice = input("Enter your choice (1 or 2): ").strip()

    if choice == '1':
        print("Please upload your Arabic book file (PDF or TXT)...")
        uploaded = files.upload()
        book_filename = list(uploaded.keys())[0]
    elif choice == '2':
        book_filename = input("Enter the path to your Arabic book file (e.g., /content/book.pdf or /content/book.txt): ").strip()
        if not book_filename:
            raise ValueError("File path cannot be empty")
    else:
        raise ValueError("Invalid choice. Please select 1 or 2.")

    # Ask about OCR for PDF files
    use_ocr = True
    if book_filename.lower().endswith('.pdf'):
        print("\nWould you like to use OCR for text extraction?")
        print("1. Yes (recommended for Arabic PDFs)")
        print("2. No (direct text extraction, may not work well for Arabic)")
        ocr_choice = input("Enter your choice (1 or 2): ").strip()
        use_ocr = True if ocr_choice == '1' else False

    # Process the book
    print(f"Processing {book_filename}...")
    chunks, text_file, json_file = process_arabic_book(book_filename, use_ocr=use_ocr)

    # Display a sample of chunks
    print("\nSample chunks:")
    for i, chunk in enumerate(chunks[:3]):
        print(f"Chunk {i}: {chunk[:100]}...")

    # Download the output files
    print("\nDownloading output files...")
    files.download(text_file)
    files.download(json_file)

else:
    # Not running in Colab
    print("Not running in Colab environment")
    book_filename = input("Enter the path to your Arabic book file (PDF or TXT): ").strip()
    if not book_filename:
        raise ValueError("File path cannot be empty")

    # Ask about OCR for PDF files
    use_ocr = True
    if book_filename.lower().endswith('.pdf'):
        print("\nWould you like to use OCR for text extraction?")
        print("1. Yes (recommended for Arabic PDFs)")
        print("2. No (direct text extraction, may not work well for Arabic)")
        ocr_choice = input("Enter your choice (1 or 2): ").strip()
        use_ocr = True if ocr_choice == '1' else False

    # Process the book
    print(f"Processing {book_filename}...")
    chunks, text_file, json_file = process_arabic_book(book_filename, use_ocr=use_ocr)

    # Display a sample of chunks
    print("\nSample chunks:")
    for i, chunk in enumerate(chunks[:3]):
        print(f"Chunk {i}: {chunk[:100]}...")

Running in Google Colab environment
Would you like to:
1. Upload the Arabic book file (PDF or TXT)
2. Enter the file path to the Arabic book file
Enter your choice (1 or 2): 2
Enter the path to your Arabic book file (e.g., /content/book.pdf or /content/book.txt): /content/selected_book.txt
Processing /content/selected_book.txt...
Extracting text from /content/selected_book.txt...
Full extracted text saved to chunked_arabic_book_full_text.txt
Processed 1099 paragraphs into 1121 chunks
Output saved to chunked_arabic_book.json

Sample chunks:
Chunk 0: بُناة الإسلام...
Chunk 1: بُناة الإسلام...
Chunk 2: محمد وخلفاؤه...



<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [8]:
!pip install sentence-transformers transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 123.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 92.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 61.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 108.2 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitl

In [9]:
import json
import torch
import numpy as np
from sentence_transformers import SentenceTransformer
from tqdm.notebook import tqdm  # For progress tracking in notebooks

# Load the model
model = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2')

# Function to generate embeddings
def generate_embeddings(texts):
    return model.encode(texts, convert_to_tensor=True)

# Load chunks from the JSON file
def load_chunks(json_file_path):
    with open(json_file_path, 'r', encoding='utf-8') as f:
        data = json.load(f)
    return data

# Process and save embeddings
def process_chunks_with_embeddings(json_file_path, output_path='chunks_with_embeddings.json'):
    # Load data
    data = load_chunks(json_file_path)

    # Extract texts from chunks
    chunk_texts = [chunk['text'] for chunk in data['chunks']]
    print(f"Generating embeddings for {len(chunk_texts)} chunks...")

    # Generate embeddings
    embeddings = generate_embeddings(chunk_texts)
    print(f"Shape of embeddings: {embeddings.shape}")

    # Convert to numpy for saving
    embeddings_numpy = embeddings.cpu().numpy()

    # Add embeddings to chunks
    for i, embedding in enumerate(embeddings_numpy):
        data['chunks'][i]['embedding'] = embedding.tolist()

    # Save the data with embeddings
    with open(output_path, 'w', encoding='utf-8') as f:
        json.dump(data, f, ensure_ascii=False, indent=2)

    # Also save embeddings separately as numpy array
    np.save('chunk_embeddings.npy', embeddings_numpy)

    print(f"Saved chunks with embeddings to {output_path}")
    print(f"Saved raw embeddings to chunk_embeddings.npy")

    return data, embeddings

try:
    from google.colab import files
    print("Running in Google Colab environment")
    print("Would you like to:")
    print("1. Upload the chunked JSON file")
    print("2. Enter the file path to the chunked JSON file")
    choice = input("Enter your choice (1 or 2): ").strip()

    if choice == '1':
        print("Please upload your chunked Arabic book JSON file...")
        uploaded = files.upload()
        json_file = list(uploaded.keys())[0]
    elif choice == '2':
        json_file = input("Enter the path to your chunked JSON file (e.g., /content/chunked_arabic_book.json): ").strip()
        if not json_file:
            raise ValueError("File path cannot be empty")
    else:
        raise ValueError("Invalid choice. Please select 1 or 2.")

    # Process the data
    data, embeddings = process_chunks_with_embeddings(json_file)

    # Download the results
    print("Downloading the files with embeddings...")
    files.download('chunks_with_embeddings.json')
    files.download('chunk_embeddings.npy')

except ImportError:
    # Not running in Colab
    print("Not running in Colab environment")
    json_file = input("Enter the path to your chunked JSON file: ").strip()
    if not json_file:
        raise ValueError("File path cannot be empty")

    # Process the data
    data, embeddings = process_chunks_with_embeddings(json_file)

# Show the first embedding as a sample
print("\nSample embedding (first 5 dimensions):")
print(embeddings[0][:5])
print(f"Total embedding dimensions: {embeddings.shape[1]}")

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/3.89k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/645 [00:00<?, ?B/s]

...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>


Sample embedding (first 5 dimensions):
tensor([ 0.2597,  0.8803, -0.5059,  0.3069, -0.0829], device='cuda:0')
Total embedding dimensions: 384


In [10]:
!pip install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 16.8 MB/s eta 0:00:00


In [11]:
!pip list | grep faiss

faiss-cpu                             1.11.0


In [21]:
import faiss
import os
import numpy as np
from sentence_transformers import SentenceTransformer

def load_embeddings(embeddings_file='chunk_embeddings.npy'):
    """Load the embeddings from the NumPy file."""
    if os.path.exists(embeddings_file):
        embeddings = np.load(embeddings_file)
        print(f"Loaded embeddings with shape: {embeddings.shape}")
        return embeddings
    else:
        raise FileNotFoundError(f"Embeddings file {embeddings_file} not found")

def load_chunks_with_embeddings(json_file='chunks_with_embeddings.json'):
    """Load chunks with their embeddings from the JSON file."""
    with open(json_file, 'r', encoding='utf-8') as f:
        data = json.load(f)
    print(f"Loaded {len(data['chunks'])} chunks with embeddings")
    return data

def create_faiss_index(embeddings, index_type='flat'):
    """
    Create a FAISS index from the embeddings.

    Parameters:
    - embeddings: numpy array of embeddings
    - index_type: type of FAISS index to create ('flat', 'ivf', or 'hnsw')

    Returns:
    - faiss index
    """
    vector_dimension = embeddings.shape[1]
    num_vectors = embeddings.shape[0]

    print(f"Creating FAISS index for {num_vectors} vectors with {vector_dimension} dimensions")

    embeddings_normalized = embeddings.copy()
    faiss.normalize_L2(embeddings_normalized)

    if index_type == 'flat':
        index = faiss.IndexFlatIP(vector_dimension)
        index.add(embeddings_normalized)

    elif index_type == 'ivf':
        nlist = int(np.sqrt(num_vectors))
        quantizer = faiss.IndexFlatIP(vector_dimension)
        index = faiss.IndexIVFFlat(quantizer, vector_dimension, nlist, faiss.METRIC_INNER_PRODUCT)
        print("Training IVF index...")
        index.train(embeddings_normalized)
        index.add(embeddings_normalized)
        index.nprobe = min(10, nlist)

    elif index_type == 'hnsw':
        M = 16
        index = faiss.IndexHNSWFlat(vector_dimension, M, faiss.METRIC_INNER_PRODUCT)
        index.add(embeddings_normalized)

    else:
        raise ValueError(f"Unknown index type: {index_type}")

    print(f"Created {index_type.upper()} index with {index.ntotal} vectors")
    return index

def save_faiss_index(index, output_file='arabic_embeddings.faiss'):
    """Save the FAISS index to a file."""
    faiss.write_index(index, output_file)
    print(f"Saved FAISS index to {output_file}")
    return output_file

def test_faiss_index(index, embeddings, chunks_data, query_text="مرحبا", top_k=5):
    """
    Test the FAISS index with a query and display results.

    Parameters:
    - index: FAISS index
    - embeddings: original embeddings
    - chunks_data: original chunks with text
    - query_text: Arabic query text
    - top_k: number of results to return
    """
    print(f"\nTesting index with query: '{query_text}'")

    model = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2')

    query_embedding = model.encode([query_text], convert_to_tensor=True)
    query_embedding_np = query_embedding.cpu().numpy()

    faiss.normalize_L2(query_embedding_np)

    distances, indices = index.search(query_embedding_np, top_k)

    print(f"\nTop {top_k} results:")
    for i, (idx, distance) in enumerate(zip(indices[0], distances[0])):
        chunk_text = chunks_data['chunks'][idx]['text']
        if len(chunk_text) > 100:
            chunk_text = chunk_text[:100] + "..."
        print(f"{i+1}. [Score: {distance:.4f}] {chunk_text}")

    return distances, indices

def main(embeddings_file='chunk_embeddings.npy',
         chunks_file='chunks_with_embeddings.json',
         index_type='flat',
         output_file='arabic_embeddings.faiss'):
    """Main function to run the indexing process."""
    embeddings = load_embeddings(embeddings_file)
    chunks_data = load_chunks_with_embeddings(chunks_file)
    index = create_faiss_index(embeddings, index_type)
    save_faiss_index(index, output_file)
    test_query = "مرحبا"
    test_faiss_index(index, embeddings, chunks_data, test_query)
    print("\nFAISS indexing complete!")
    return index, chunks_data

# For Google Colab: Check if running in Colab and handle file input
try:
    from google.colab import files
    RUNNING_IN_COLAB = True
    print("Running in Google Colab environment")

    print("Would you like to:")
    print("1. Upload the embeddings and chunks files")
    print("2. Enter the file paths to the embeddings and chunks files")
    choice = input("Enter your choice (1 or 2): ").strip()

    if choice == '1':
        print("Please upload chunk_embeddings.npy and chunks_with_embeddings.json...")
        uploaded = files.upload()
        if 'chunk_embeddings.npy' not in uploaded or 'chunks_with_embeddings.json' not in uploaded:
            raise ValueError("Both chunk_embeddings.npy and chunks_with_embeddings.json must be uploaded")
        embeddings_file = 'chunk_embeddings.npy'
        chunks_file = 'chunks_with_embeddings.json'
    elif choice == '2':
        embeddings_file = input("Enter path to embeddings file (e.g., /content/chunk_embeddings.npy): ").strip() or "chunk_embeddings.npy"
        chunks_file = input("Enter path to chunks file (e.g., /content/chunks_with_embeddings.json): ").strip() or "chunks_with_embeddings.json"
        if not embeddings_file or not chunks_file:
            raise ValueError("File paths cannot be empty")
    else:
        raise ValueError("Invalid choice. Please select 1 or 2.")

    # Choose index type
    print("\nChoose FAISS index type:")
    print("1. Flat index (exact search, slower but most accurate)")
    print("2. IVF index (approximate search, good balance of speed and accuracy)")
    print("3. HNSW index (very fast, good accuracy, best for large datasets)")
    choice = input("Enter your choice (1/2/3): ").strip()

    index_type = {
        '1': 'flat',
        '2': 'ivf',
        '3': 'hnsw'
    }.get(choice, 'flat')

    # Run indexing
    index, chunks_data = main(embeddings_file, chunks_file, index_type)

    # Download the index
    print("Downloading the FAISS index file...")
    files.download('arabic_embeddings.faiss')

except ImportError:
    RUNNING_IN_COLAB = False
    print("Not running in Colab environment")

    embeddings_file = input("Enter path to embeddings file (default: chunk_embeddings.npy): ").strip() or "chunk_embeddings.npy"
    chunks_file = input("Enter path to chunks file (default: chunks_with_embeddings.json): ").strip() or "chunks_with_embeddings.json"
    if not embeddings_file or not chunks_file:
        raise ValueError("File paths cannot be empty")

    print("\nChoose FAISS index type:")
    print("1. Flat index (exact search, slower but most accurate)")
    print("2. IVF index (approximate search, good balance of speed and accuracy)")
    print("3. HNSW index (very fast, good accuracy, best for large datasets)")
    choice = input("Enter your choice (1/2/3): ").strip()

    index_type = {
        '1': 'flat',
        '2': 'ivf',
        '3': 'hnsw'
    }.get(choice, 'flat')

    index, chunks_data = main(embeddings_file, chunks_file, index_type)

Running in Google Colab environment
Would you like to:
1. Upload the embeddings and chunks files
2. Enter the file paths to the embeddings and chunks files
Enter your choice (1 or 2): 2
Enter path to embeddings file (e.g., /content/chunk_embeddings.npy): /content/chunk_embeddings.npy
Enter path to chunks file (e.g., /content/chunks_with_embeddings.json): /content/chunks_with_embeddings.json

Choose FAISS index type:
1. Flat index (exact search, slower but most accurate)
2. IVF index (approximate search, good balance of speed and accuracy)
3. HNSW index (very fast, good accuracy, best for large datasets)
Enter your choice (1/2/3): 3
Loaded embeddings with shape: (1121, 384)
Loaded 1121 chunks with embeddings
Creating FAISS index for 1121 vectors with 384 dimensions
Created HNSW index with 1121 vectors
Saved FAISS index to arabic_embeddings.faiss

Testing index with query: 'مرحبا'

Top 5 results:
1. [Score: 0.8315] أهل جاوة
2. [Score: 0.7155] في منى
3. [Score: 0.6919] أيها المبعوثُ فينا


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [25]:

from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [26]:
!cp -r /content/arabic_embeddings.faiss /content/drive/MyDrive/
!cp -r /content/chunks_with_embeddings.json /content/drive/MyDrive/
!cp -r /content/chunk_embeddings.npy /content/drive/MyDrive/
!cp -r /content/chunked_arabic_book.json /content/drive/MyDrive/

In [13]:
!pip install rank_bm25

In [22]:
import faiss
import numpy as np
import json
from sentence_transformers import SentenceTransformer
from rank_bm25 import BM25Okapi
import re

# Load FAISS index
index = faiss.read_index('/content/arabic_embeddings.faiss')

# Load chunk data
with open('/content/chunks_with_embeddings.json', 'r', encoding='utf-8') as f:
    chunks_data = json.load(f)
chunks = chunks_data['chunks']  # List of dictionaries with 'text' key

# Initialize the embedding model
model = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2')

# Prepare BM25 index
def tokenize_arabic(text):
    """Tokenize Arabic text by splitting on whitespace and removing punctuation."""
    # Remove punctuation and split
    tokens = re.findall(r'\b\w+\b', text)
    return tokens

# Create tokenized corpus for BM25
tokenized_corpus = [tokenize_arabic(chunk['text']) for chunk in chunks]
bm25 = BM25Okapi(tokenized_corpus)

def encode_query(query):
    """Encode an Arabic query into an embedding."""
    embedding = model.encode([query], convert_to_tensor=False)
    return embedding[0]

def search_index(query_embedding, k=5):
    """Search the FAISS index for top-k similar chunks."""
    query_embedding = query_embedding / np.linalg.norm(query_embedding)
    query_embedding = query_embedding.reshape(1, -1).astype('float32')
    distances, indices = index.search(query_embedding, k)
    return distances[0], indices[0]

def retrieve_bm25(query, k=5):
    """Retrieve top-k chunks based on BM25 scoring."""
    tokenized_query = tokenize_arabic(query)
    scores = bm25.get_scores(tokenized_query)
    # Get top-k indices
    top_k_indices = np.argsort(scores)[::-1][:k]
    results = []
    for idx in top_k_indices:
        chunk_text = chunks[idx]['text']
        score = scores[idx]
        results.append((chunk_text, score))
    return results

def retrieve_semantic(query, k=5):
    """Retrieve top-k chunks based on semantic similarity."""
    query_embedding = encode_query(query)
    distances, indices = search_index(query_embedding, k)
    results = []
    for dist, idx in zip(distances, indices):
        chunk_text = chunks[idx]['text']
        results.append((chunk_text, dist))
    return results


In [23]:
الexit_program = False

while not exit_program:
    query = input("أدخل استفسارك بالعربية (Enter your Arabic query): ")
    k = 5  # Number of top results to show

    # Get results
    bm25_results = retrieve_bm25(query, k)
    semantic_results = retrieve_semantic(query, k)

    # Display results
    print("\nنتائج البحث:\n" + "=" * 80)
    for i in range(k):
        print(f"{i+1}.")

        if i < len(bm25_results):
            bm25_text, bm25_score = bm25_results[i]
            print(f"   [BM25: {bm25_score:.4f}] {bm25_text}")
        else:
            print("   [BM25: ---] لا توجد نتيجة")

        if i < len(semantic_results):
            sem_text, sem_score = semantic_results[i]
            print(f"   [Semantic: {sem_score:.4f}] {sem_text}")
        else:
            print("   [Semantic: ---] لا توجد نتيجة")

        print("-" * 80)

    # Ask user to continue or exit
    while True:
        continue_response = input("هل تريد إجراء استفسار آخر؟ (ن/ل): ").strip().lower()
        if continue_response.startswith('ن') or continue_response.startswith('y'):
            break  # Continue
        elif continue_response.startswith('ل') or continue_response.startswith('n'):
            exit_program = True
            break
        else:
            print("الرجاء إدخال 'ن' للمتابعة أو 'ل' للإنهاء.")


أدخل استفسارك بالعربية (Enter your Arabic query): خلافة عمر بن الخطاب

نتائج البحث:
1.
   [BM25: 13.1908] خلافة عثمان بن عفان
   [Semantic: 0.8203] سياسة علي
--------------------------------------------------------------------------------
2.
   [BM25: 13.0384] لقد سمعته بالأمس يقول: «اللهم أيِّد الإسلام بأبي الحكم بن هشام، أو بعمر بن الخطاب. » فالله، الله يا عمر.
   [Semantic: 0.7670] فقه علي وزهده
--------------------------------------------------------------------------------
3.
   [BM25: 12.9843] خلافة أمير المؤمنين عمر
   [Semantic: 0.7506] قتل عمر
--------------------------------------------------------------------------------
4.
   [BM25: 9.8280] حقًّا، إن رسول الله ﷺ ما كان ينطق عن الهوى، إن هو إلا وحيٌ يوحى، علَّمه شديد القوى، وقد دعا ربه أن يعز الإسلام بأحد العمرين، فكان أن أسلم عمر بن الخطاب.
   [Semantic: 0.7497] خلافة عثمان بن عفان
--------------------------------------------------------------------------------
5.
   [BM25: 9.5772] خلافة علي
   [Semantic: 0.7353] »
--------